### Task: Automatically build abstracts of text documents.

Input: An array of texts in JSON format. Sample texts are at example_texts.json.

Output: An array of abstracts in JSON format (the order of abstracts corresponds to the order of texts in the input data).

The maximum size of each of the abstracts is 300 characters (including white space). If the size of the abstract exceeds the specified threshold, then only the first 300 characters will be evaluated. A trivial solution (the first 300 characters of the document) is allowed, but not welcome.

You can briefly describe the solution in the first line of the download file after the # character. The information will be useful to the authors of the course to gain an idea of the methods and approaches used.

Rating: ROUGE-2 - proximity to a set of manually compiled abstracts based on bigrams of words (value from 0 to 1).

Input can be found at dataset_43428_1.txt

Output is at outdataset_43428_1.txt

Solution description:


## Test part

In [1]:
import pandas as pd

df_jsexample = pd.read_json('example_texts.json')
df_jsexample

,0
0,Увидеть мысль\n\nЯпонским ученым удалось распо...
1,"Карбофос\n\nКарбофос ( O , O - Диметил - S - (..."
2,Киев идет на уступки : Яценюк передает власть ...
3,« Лужков . Итоги »\nКонец эры хозяйственника и...
4,... в ходе написания ходатайства : сделать его...


In [2]:
sample_text = df_jsexample[0][0]
sample_text

'Увидеть мысль\n\nЯпонским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг .\n\nКак пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение , увиденное человеком , опираясь только на сканирование электрических сигналов мозга .\n\nВ ходе эксперимента исследователи продемонстрировали испытуемым серию из 440 различных картинок , представлявших собой произвольное сочетание темных и светлых пятен , на 100-пиксельном экране .\nВ процессе демонстрации измерялась активность в зрительной коре головного мозга с помощью функционального магниторезонансного сканера .\nПосле этого испытуемым были показаны буквы , составляющие слово neuron , и элементарные геометрические фигуры .\nНа основе сравнения предыдущей серии изображений и показаний сканера исследователям удалось правильно реконструировать новые изображения по сигналам мозговой ак

In [3]:
from gensim.summarization import summarize
from nltk import tokenize
import re

def simpleword_count(text):
    word_list = tokenize.word_tokenize(text)
    return len(word_list)
'''
Functions for Dataframe changing: start
'''
def word_count_df(row):
    text = row[0]
    word_list = tokenize.word_tokenize(text)
    return len(word_list)


def clean_text(row):
    text = row[0]
    text = text.lower()
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    return text

def summary_text(row):
    text = row['clean_text']
    sentences = tokenize.sent_tokenize(text)
    wc = simpleword_count(text)
    if len(sentences) == 1:
        return text
    if wc <= 1500:
        summarized_text = summarize(text, ratio=0.2, split=False)
    elif wc >= 1500:
        summarized_text = summarize(text, word_count=300, split=False)
    
    return summarized_text

def summary_word_count_df(row):
    text = row['summary_text']
    word_list = tokenize.word_tokenize(text)
    return len(word_list)

'''
Functions for Dataframe changing: end
'''

'\nFunctions for Dataframe changing: end\n'

In [4]:
df_jsexample['orig_word_count'] = df_jsexample.apply(lambda row: word_count_df(row), axis=1)

In [5]:
df_jsexample['clean_text'] = df_jsexample.apply(lambda row: clean_text(row), axis=1)

In [6]:
df_jsexample['summary_text'] = df_jsexample.apply(lambda row: summary_text(row), axis=1)

In [7]:
df_jsexample['sum_word_count'] = df_jsexample.apply(lambda row: summary_word_count_df(row), axis=1)

In [8]:
df_jsexample.head()

,0,orig_word_count,clean_text,summary_text,sum_word_count
0,Увидеть мысль\n\nЯпонским ученым удалось распо...,155,увидеть мысль японским ученым удалось распозна...,"как пишет yomiuri , группе сотрудников отделен...",34
1,"Карбофос\n\nКарбофос ( O , O - Диметил - S - (...",131,"карбофос карбофос ( o , o - диметил - s - ( 1,...",в настоящее время вытеснен более совершенными ...,12
2,Киев идет на уступки : Яценюк передает власть ...,282,киев идет на уступки : яценюк передает власть ...,киев идет на уступки : яценюк передает власть ...,38
3,« Лужков . Итоги »\nКонец эры хозяйственника и...,2772,« лужков . итоги » конец эры хозяйственника ил...,пока борис немцов подсчитывал разницу между ст...,279
4,... в ходе написания ходатайства : сделать его...,323,... в ходе написания ходатайства : сделать его...,"закончим , как и положено : &quot; на основани...",110


## Task part

In [9]:
df_dataset = pd.read_json('dataset_43428_1.txt')
df_dataset

,0
0,В Сербии арестован последний военный преступни...
1,АнтиПоттер\nРуководство по эксплуатации волшеб...
2,Авто по карману\nМихаил Прохоров собирается за...
3,В Казахстане состоялись выборы Президента\n\nВ...
4,В Москве подвели итоги Международной конференц...
...,...
195,13-я экспедиция МКС\n\n30 марта 2006 года с ко...
196,Пираты не признали вину\n\nВ американском горо...
197,Аллегория удачи .\nВизит Дмитрия Медведева в К...
198,Тем временем участницы панк-группы встретились...


In [10]:
df_dataset['orig_word_count'] = df_dataset.apply(lambda row: word_count_df(row), axis=1)

In [11]:
df_dataset['clean_text'] = df_dataset.apply(lambda row: clean_text(row), axis=1)

In [12]:
df_dataset['summary_text'] = df_dataset.apply(lambda row: summary_text(row), axis=1)

In [13]:
df_dataset['summary_word_count'] = df_dataset.apply(lambda row: summary_word_count_df(row), axis=1)

In [14]:
df_dataset

,0,orig_word_count,clean_text,summary_text,summary_word_count
0,В Сербии арестован последний военный преступни...,213,в сербии арестован последний военный преступни...,"за информацию , способную помочь в поимке гора...",50
1,АнтиПоттер\nРуководство по эксплуатации волшеб...,1918,антипоттер руководство по эксплуатации волшебн...,мне уже говорили мои молодые коллеги и просто ...,296
2,Авто по карману\nМихаил Прохоров собирается за...,110,авто по карману михаил прохоров собирается зан...,интересы у олигарха михаила прохорова самые ра...,18
3,В Казахстане состоялись выборы Президента\n\nВ...,376,в казахстане состоялись выборы президента в во...,по предварительным данным опроса избирателей н...,51
4,В Москве подвели итоги Международной конференц...,880,в москве подвели итоги международной конференц...,в москве подвели итоги международной конференц...,228
...,...,...,...,...,...
195,13-я экспедиция МКС\n\n30 марта 2006 года с ко...,576,13-я экспедиция мкс 30 марта 2006 года с космо...,13-я экспедиция мкс 30 марта 2006 года с космо...,162
196,Пираты не признали вину\n\nВ американском горо...,253,пираты не признали вину в американском городе ...,"кроме того , в руках вмс сша у берегов сомали ...",66
197,Аллегория удачи .\nВизит Дмитрия Медведева в К...,1136,аллегория удачи . визит дмитрия медведева в ки...,"трудно сказать , смирились ли сторонники викто...",344
198,Тем временем участницы панк-группы встретились...,519,тем временем участницы панк-группы встретились...,тем временем участницы панк-группы встретились...,100


In [15]:
df_dataset.to_json('gensim_output_dataset.json')

In [16]:
out_dataset = pd.DataFrame(df_dataset['summary_text'])
out_dataset

,summary_text
0,"за информацию , способную помочь в поимке гора..."
1,мне уже говорили мои молодые коллеги и просто ...
2,интересы у олигарха михаила прохорова самые ра...
3,по предварительным данным опроса избирателей н...
4,в москве подвели итоги международной конференц...
...,...
195,13-я экспедиция мкс 30 марта 2006 года с космо...
196,"кроме того , в руках вмс сша у берегов сомали ..."
197,"трудно сказать , смирились ли сторонники викто..."
198,тем временем участницы панк-группы встретились...


In [17]:

# output file can be read only with pandas. 
# example of encoded string ["\u0412 \u0421\u0435\u0440\u0431\u0438\u0438
# How to encode russian text properly in json?
out_dataset.to_json('outdataset_43428_1.txt', orient='values')

In [18]:
out_dataset.to_csv('outdataset_43428_1.csv',header=False, index=False)